In [1]:
environment_library = 'gym'

In [2]:
import IPython
import time
from acme import environment_loop
from acme import specs
from acme import wrappers
from acme.agents.tf import d4pg
from acme.tf import networks
from acme.tf import utils as tf2_utils
from acme.utils import loggers
import numpy as np
import sonnet as snt

# Import the selected environment lib
if environment_library == 'dm_control':
    from dm_control import suite
elif environment_library == 'gym':
    import gym

# Imports required for visualization
import pyvirtualdisplay
import imageio
import base64

# Set up a virtual display for rendering.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()


/home/sean/Documents/venvs/dm_acme/acme/lib/python3.8/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/sean/Documents/venvs/dm_acme/acme/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/home/sean/Documents/venvs/dm_acme/acme/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/home/sean/Documents/venvs/dm_acme/acme/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
 

In [3]:
if environment_library == 'dm_control':
    environment = suite.load('cartpole', 'balance')

elif environment_library == 'gym':
    environment = gym.make('Pendulum-v1')
    environment = wrappers.GymWrapper(environment)  # To dm_env interface.

else:
    raise ValueError(
      "Unknown environment library: {};".format(environment_library) +
      "choose among ['dm_control', 'gym'].")

# Make sure the environment outputs single-precision floats.
environment = wrappers.SinglePrecisionWrapper(environment)

# Grab the spec of the environment.
environment_spec = specs.make_environment_spec(environment)


In [4]:
#@title Build agent networks

# Get total number of action dimensions from action spec.
num_dimensions = np.prod(environment_spec.actions.shape, dtype=int)

# Create the shared observation network; here simply a state-less operation.
observation_network = tf2_utils.batch_concat

# Create the deterministic policy network.
policy_network = snt.Sequential([
    networks.LayerNormMLP((256, 256, 256), activate_final=True),
    networks.NearZeroInitializedLinear(num_dimensions),
    networks.TanhToSpec(environment_spec.actions),
])

# Create the distributional critic network.
critic_network = snt.Sequential([
    # The multiplexer concatenates the observations/actions.
    networks.CriticMultiplexer(),
    networks.LayerNormMLP((512, 512, 256), activate_final=True),
    networks.DiscreteValuedHead(vmin=-150., vmax=150., num_atoms=51),
])

# Create a logger for the agent and environment loop.
agent_logger = loggers.TerminalLogger(label='agent', time_delta=10.)
env_loop_logger = loggers.TerminalLogger(label='env_loop', time_delta=10.)

# Create the D4PG agent.
agent = d4pg.D4PG(
    environment_spec=environment_spec,
    policy_network=policy_network,
    critic_network=critic_network,
    observation_network=observation_network,
    sigma=1.0,
    logger=agent_logger,
    checkpoint=False
)

# Create an loop connecting this agent to the environment created above.
env_loop = environment_loop.EnvironmentLoop(
    environment, agent, logger=env_loop_logger)



[reverb/cc/platform/tfrecord_checkpointer.cc:150]  Initializing TFRecordCheckpointer in /tmp/tmp7hs69vyj.
[reverb/cc/platform/tfrecord_checkpointer.cc:386] Loading latest checkpoint from /tmp/tmp7hs69vyj
[reverb/cc/platform/default/server.cc:71] Started replay server on port 23474
2022-04-06 16:51:15.454631: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [5]:
# Run a `num_episodes` training episodes.
# Rerun this cell until the agent has learned the given task.
tic = time.perf_counter()
env_loop.run(num_episodes=1000)
toc = time.perf_counter()

print ("time elapsed (s): {}".format(toc - tic))

[reverb/cc/client.cc:165] Sampler and server are owned by the same process (30550) so Table priority_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (30550) so Table priority_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (30550) so Table priority_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (30550) so Table priority_table is accessed directly without gRPC.


time elapsed (s): 312.5092550870013


In [6]:


# Create a simple helper function to render a frame from the current state of
# the environment.
if environment_library == 'dm_control':
    def render(env):
        return env.physics.render(camera_id=0)
elif environment_library == 'gym':
    def render(env):
        return env.environment.render(mode='rgb_array')
else:
    raise ValueError(
      "Unknown environment library: {};".format(environment_library) +
      "choose among ['dm_control', 'gym'].")

def display_video(frames, filename='temp.mp4'):
    """Save and display video."""

  # Write video
    with imageio.get_writer(filename, fps=60) as video:
        for frame in frames:
            video.append_data(frame)

  # Read video and display the video
    video = open(filename, 'rb').read()
    b64_video = base64.b64encode(video)
    video_tag = ('<video  width="320" height="240" controls alt="test" '
               'src="data:video/mp4;base64,{0}">').format(b64_video.decode())

    return IPython.display.HTML(video_tag)



In [7]:


timestep = environment.reset()
frames = [render(environment)]

while not timestep.last():
    # Simple environment loop.
    action = agent.select_action(timestep.observation)
    timestep = environment.step(action)

    # Render the scene and add it to the frame stack.
    frames.append(render(environment))

# Save and display a video of the behaviour.
display_video(np.array(frames))



[swscaler @ 0x6850680] Warning: data is not aligned! This can lead to a speed loss
